In [403]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_lg")
import dateparser
import pygwalker as pyg

In [516]:

#sentence = "send a reminder for the meeting at 5 pm to everyone now and create a document for meeting notes then send a message to Rajesh sir."
#sentence = "send me a reminder on mail regarding the meeting tomorrow"
sentence = "add a meeting with rajesh sir for tomorrow 5 pm in my schedule"
doc = nlp(sentence)
graph = {}


def generateTable(doc):

    token_text = []
    token_lemma = []
    token_pos = []
    token_tag = []
    token_dep = []
    token_children =[]
    times = []
    dates = []


    
    
    #get root_node 
    
    for token in doc:
        token_text.append(token.text)
        token_lemma.append(token.lemma_ )
        token_pos.append(token.pos_) 
        token_tag.append(token.tag_)
        token_dep.append(token.dep_)
        token_children.append([child.lemma_ for child in token.children])
        graph[token] = token.children
        if token.dep_=="ROOT":
            root_node = token

    df_dict = {
        "text":token_text,
        "lemma":token_lemma,
        "pos":token_pos,
        "dep":token_dep,
        "tag":token_tag,
        "children":token_children
    }   


#     get date time
    for ent in doc.ents:
            print(ent,ent.label_)
            if ent.label_ in ["TIME", "DATE"]:
                if ent.label_ == "TIME":
                    times.append(ent.text)
                elif ent.label_ == "DATE":
                    dates.append(ent.text)
    print(dates,times)

    df = pd.DataFrame(df_dict)

    return df,root_node

df , root_node = generateTable(doc)
df

tomorrow 5 pm TIME
[] ['tomorrow 5 pm']


,text,lemma,pos,dep,tag,children
0,add,add,VERB,ROOT,VB,"[meeting, for, in]"
1,a,a,DET,det,DT,[]
2,meeting,meeting,NOUN,dobj,NN,"[a, with]"
3,with,with,ADP,prep,IN,[sir]
4,rajesh,rajesh,ADJ,amod,JJ,[]
5,sir,sir,NOUN,pobj,NN,[rajesh]
6,for,for,ADP,prep,IN,[tomorrow]
7,tomorrow,tomorrow,NOUN,pobj,NN,[pm]
8,5,5,NUM,nummod,CD,[]
9,pm,pm,NOUN,appos,NN,[5]


Find the action in the sentence

In [517]:
from transformers import pipeline
qa_model = pipeline("question-answering")


def break_querry(sentence):
    
    
    #return dict
    
    answer = {}
    
    #simplify the sentence and retrieve the root, action
    
    doc = nlp(sentence)
    simple_sentence = ""
    action = ""
    start = False
    
    for token in doc:
        if token.tag_ == "VBP":
            continue

        elif token.dep_=="ROOT":
            root_node = token
            start = True
            action+=" "+token.lemma_

            for t in token.children:

                if t.pos_=="NOUN" and t.dep_!="punct":
                    action+=" "+t.lemma_
                    got_action = True
                    break
        
        if start:
            simple_sentence+=" "+token.text


    sentence = simple_sentence.strip()
    action = action.strip()
    
    print("Simplified Sentence :",sentence)
    
    function =  find_functionality(action)[1]
    print("Functionlaity:",function)
    answer["FUNCTION"]=function
    
    print("Action :",action)
    answer["ACTION"]=action
    
    #find the target
    
    stack = [root_node]
    visited=[root_node]
    target_people = []
    while stack:
        token = stack.pop()
        
        for i in token.children:
            if i.pos_ in ["PRON","PROPN","ADP"] and i not in visited:
                stack.append(i)
                visited.append(i)
                if i.pos_ in ["PRON","PROPN"] :
                    target_people.append(i.lemma_)
                

    print("Who :",target_people)
    answer["WHO"]=target_people
    
    context = sentence
    
    
    #get what
    
    scheduled_for = " ".join(target_people)
    q1 = "What should I "+action +"for?"
    w1 = qa_model(question = q1, context = context)
    print("What:",w1['answer'])
    answer["WHAT"]=w1['answer']
    
    
    #get when parsed from date time

    dates = []
    times = []
    for ent in doc.ents:

        if ent.label_ in ["TIME", "DATE"]:
            if ent.label_ == "TIME":
                times.append(ent.text)
            elif ent.label_ == "DATE":
                dates.append(ent.text)

    date = "".join(dates)
    time = "".join(times)
    input_text = date + " " + time
    parsed_datetime = dateparser.parse(input_text)
    print("When:",parsed_datetime)
    answer["WHEN"]=parsed_datetime

    
    #get where

    q4 = "Where should I "+action 
    w4 = qa_model(question = q4, context = context)
    if w4["score"]>0.5:
        w4 = w4['answer']
    else:
        w4 = None

    print("Where:",w4)
    answer["WHERE"]=w4
    
    return answer
    
ans = break_querry(sentence)
print(ans)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Simplified Sentence : add a meeting with rajesh sir for tomorrow 5 pm in my schedule
Functionlaity: ADD TO SCHEDULE
Action : add meeting
Who : []
What: a meeting with rajesh sir
When: 2024-03-17 17:00:00
Where: None
{'FUNCTION': 'ADD TO SCHEDULE', 'ACTION': 'add meeting', 'WHO': [], 'WHAT': 'a meeting with rajesh sir', 'WHEN': datetime.datetime(2024, 3, 17, 17, 0), 'WHERE': None}


In [506]:
functionality_space = {
    "add reminder":"ADD MESSAGE",
    "add message":"ADD MESSAGE",
    "tell":"ADD MESSAGE",
    "send message":"SEND MESSAGE",
    "drop message":"ADD MESSAGE",
    "have message":"GET MESSAGE",
    "remind":"ADD MESSAGE",
    "leave message":"ADD MESSAGE",
    "add":"ADD",
    "create list":"CREATE LIST",
    "add event":"ADD TO SCHEDULE",
    "schedule event":"ADD TO SCHEDULE",
    "add expense":"REMOVE FUND",
    "add fund":"ADD FUND",
    "remove fund":"REMOVE FUND",
    "spend":"REMOVE FUND"
    }



In [501]:
def find_functionality(action):
    best_ans = (0,0)

    for act in functionality_space:
        doc1 = nlp(act)
        doc2 = nlp(action)
        similarity = doc1.similarity(doc2)
        # print(data, similarity)
        if similarity > best_ans[0]:
            best_ans = (similarity,functionality_space[act])

    return best_ans

function =  find_functionality(action)[1]
function

'ADD MESSAGE'

In [476]:
known_person = [
    "I",
    "everyone",
    "mashaal",
    "priyansh",
    "anmol",
    "divyansh",
    "rupesh",
    "rajesh",
    "ashutosh",
    "naveen",
    "harshit",
    "aswin",
    "herchelle",
]

In [477]:
known_objects = [
    "messsage",
    "schedule",
    "inventory",
    "funds",
    "list"
]

In [437]:
def find_known_names(sentence, known_names):
    found_names = []
    doc = nlp(sentence)
    
    for token in doc:
        closest_match = difflib.get_close_matches(token.lemma_, known_names, n=1, cutoff=0.7)
        if closest_match:
            found_names.append((token.lemma_, closest_match[0]))
    return found_names

Check for W words
Get action
find the person WHO
find WHAT 

In [438]:
from transformers import pipeline
qa_model = pipeline("question-answering")

def get_details(action,sentence,function):
    
    
    #parse the sentence

    print('Action:',action)
    
    
    doc = nlp(sentence)

    #detect objects in the sentece 
    objects = []
    for token in doc:
        if token.pos_ in ["PROPN","PRON"]:
            objects.append(token.lemma_)
    
    
    context = sentence


    #get whom for checking known people
    
    q2 = "Whom should I "+action +" to?"
    w2 = qa_model(question = q2, context = context)
    print(w2)
    

    found_names = find_known_names(sentence, known_person)
    
    scheduled_for=found_names[0][0] if found_names else ""
    print(found_names)
    print(scheduled_for)

    #what
    q1 = "What should I "+action +" to "+scheduled_for
    w1 = qa_model(question = q1, context = context)



    print("What:",w1['answer'])




    #get when parsed from date time

    dates = []
    times = []
    for ent in doc.ents:

        if ent.label_ in ["TIME", "DATE"]:
            if ent.label_ == "TIME":
                times.append(ent.text)
            elif ent.label_ == "DATE":
                dates.append(ent.text)

    date = "".join(dates)
    time = "".join(times)
    input_text = date + " " + time
    parsed_datetime = dateparser.parse(input_text)
    print("When:",parsed_datetime)


    #get where

    q4 = "Where should I "+action +" "+scheduled_for+" to" + w1['answer']
    w4 = qa_model(question = q4, context = context)
    if w4["score"]>0.5:
        w4 = w4['answer']
    else:
        w4 = None

    print("Where:",w4)

        

    
get_details(action ,sentence,function )

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Action: send reminder
{'score': 0.8457335233688354, 'start': 53, 'end': 63, 'answer': 'Rajesh sir'}
[('I', 'I'), ('rupesh', 'rupesh'), ('Rajesh', 'rajesh')]
I
What: the meeting with Rajesh sir
When: 2024-03-15 22:53:14.783255
Where: None


In [295]:
import difflib


# Example usage:
known_names = ['John', 'Alice', 'Bob', 'Charlie', 'David', 'Emily']
sentence = "Hello, my name is Jhon and I'm talking to Alica."
found_names = find_known_names(sentence, known_names)
print("Found names with nearest spelling:")
for original, closest in found_names:
    print(f"Original: {original}, Closest: {closest}")

Found names with nearest spelling:
Original: Jhon, Closest: John
Original: Alica., Closest: Alice


In [321]:
def find_known_names(sentence, known_names):
    found_names = []
    for word in sentence.split():
        closest_match = difflib.get_close_matches(word, known_names, n=1, cutoff=0.7)
        if closest_match:
            found_names.append((word, closest_match[0]))
    return found_names

found_names = find_known_names(sentence, known_person)
print(found_names)

[('Ashwin', 'aswin'), ('Rajesh', 'rajesh')]


In [203]:
q2 = "Who should I remind"
qa_model(question = q2, context  = sentence)

{'score': 0.4729948341846466, 'start': 28, 'end': 36, 'answer': 'tomorrow'}

In [117]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_lg")

# Define a function to extract action and information from a sentence
def extract_action_and_info(sentence):
    # Process the input sentence using spaCy
    doc = nlp(sentence)
    
    # Initialize variables to store action and information
    action = None
    information = {}
    
    # Iterate over each token in the processed sentence
    for token in doc:
        # Check if the token is a verb (indicating an action)
        if token.pos_ == "VERB":
            action = token.lemma_  # Use the lemma of the verb as the action
        # Check if the token is a noun or proper noun (indicating information)
        elif token.pos_ in ["NOUN", "PROPN","PRON"]:
            information[token.text] = token.tag_  # Store the noun and its part of speech tag
    
    return action, information

# Example usage
sentence = "send a reminder to me about the meeting tomorrow with Rajesh sir"
action, information = extract_action_and_info(sentence)
print("Action:", action)
print("Information:")
for noun, pos_tag in information.items():
    print(f"{noun}: {pos_tag}")


Action: send
Information:
reminder: NN
me: PRP
meeting: NN
tomorrow: NN
Rajesh: NNP
sir: NNP


In [141]:
from transformers import pipeline
qa_model = pipeline("question-answering")
info = "tell Priyansh to talk to Naveen sir"

qa_model(question = "what do I tell Priyansh", context = info)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.6087073683738708,
 'start': 14,
 'end': 31,
 'answer': 'to talk to Naveen'}

In [194]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract action and information from a sentence
def extract_action_and_info(sentence):
    # Process the input sentence using spaCy
    doc = nlp(sentence)
    
    # Initialize variables to store action, subject, object, time, and other information
    action = None
    subject = None
    obj = None
    time = None
    other_info = {}
    
    # Iterate over each token in the processed sentence
    for token in doc:
        # Check if the token is a verb (indicating an action)
        if token.pos_ == "VERB":
            action = token.lemma_  # Use the lemma of the verb as the action
        # Check if the token is a noun or proper noun (indicating subject or object)
        elif token.pos_ in ["NOUN", "PROPN"]:
            # Check if the token is a subject (nsubj) or object (dobj)
            if token.dep_ == "nsubj":
                subject = token.text
            elif token.dep_ == "dobj":
                obj = token.text
        # Check if the token is a time expression
        elif token.ent_type_ == "DATE":
            time = token.text
        # Store other relevant information
        else:
            other_info[token.text] = token.tag_
    
    return action, subject, obj, time, other_info

# Example usage
sentence = "send me a reminder message at 12pm today about the meeting tomorrow with Rajesh sir "
action, subject, obj, time, other_info = extract_action_and_info(sentence)
print("Action:", action)
print("Subject:", subject)
print("Object:", obj)
print("Time:", time)
print("Other Information:", other_info)


Action: send
Subject: None
Object: message
Time: None
Other Information: {'me': 'PRP', 'a': 'DT', 'at': 'IN', '12': 'CD', 'about': 'IN', 'the': 'DT', 'with': 'IN'}
